In [75]:
import pandas as pd
import numpy as np
import datetime
import pickle
from string import Template
import matplotlib.pyplot as plt
%matplotlib inline

In [76]:
with open('MTA_Apr_June_with_weeks.pickle', 'rb') as f:
    data = pickle.load(f)

#remove white space from columns

In [77]:

data.columns = [column.strip() for column in data.columns]
data.rename(columns={'week':'WEEK'}, inplace=True)
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,WEEK
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/21/2018,00:00:00,REGULAR,6590024,2232650,1
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/21/2018,04:00:00,REGULAR,6590038,2232663,1
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/21/2018,08:00:00,REGULAR,6590050,2232693,1
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/21/2018,12:00:00,REGULAR,6590131,2232766,1
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/21/2018,16:00:00,REGULAR,6590350,2232816,1


In [78]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1970076 entries, 0 to 196689
Data columns (total 12 columns):
C/A         object
UNIT        object
SCP         object
STATION     object
LINENAME    object
DIVISION    object
DATE        object
TIME        object
DESC        object
ENTRIES     int64
EXITS       int64
WEEK        int64
dtypes: int64(3), object(9)
memory usage: 195.4+ MB


In [79]:
#that's a lot of data! let's drop some observations off the bat. Getting rid of observations from 0:00 and 4:00, 
#    leaving only 12:00 pm-8:00 pm, when most tech-employees and potential donors might be commuting home or getting lunch
data = data[data['TIME'].isin(('12:00:00','16:00:00', '20:00:00'))]
data.shape

(509863, 12)

In [80]:
#function to calculate the difference between an observation's max and min values, which will mostly occur at 12:00 and 8:.
#will be applied on unique turnstile_id + day-level
def difference(srs):
    return srs.max() - srs.min()

#Identifying and grouping by unique turnstiles, taking the daily max-daily min for that turnstile:

In [81]:
turnstile_col_names =['STATION','C/A','UNIT','SCP']

by_turnstile_date =  data.groupby(by=turnstile_col_names+['WEEK']+['DATE'], as_index=False).agg(difference)
by_turnstile_date['turnstile_id'] = by_turnstile_date.apply(lambda x: str(x['STATION']+'_'+x['C/A']+'_'+x['UNIT']+'_'+x['SCP']), axis=1)

In [82]:
by_turnstile_date['ENTRIES'].describe()
#plt.hist(by_station_date['ENTRIES'][by_station_date['ENTRIES']>861])

count    1.702130e+05
mean     5.312254e+04
std      8.530321e+06
min      0.000000e+00
25%      1.700000e+02
50%      4.540000e+02
75%      8.610000e+02
max      1.953723e+09
Name: ENTRIES, dtype: float64

#turnstiles with 1 really high day's entries probably are showing data anomalies:

In [200]:
by_turnstile_date[by_turnstile_date['ENTRIES']>5000].turnstile_id.value_counts()

GRD CNTRL-42 ST_R238_R046_00-03-04     18
59 ST COLUMBUS_N051_R084_02-00-01       2
AVENUE H_B020_R263_00-03-01             1
174-175 STS_N212_R253_01-06-01          1
CASTLE HILL AV_R418_R106_00-03-00       1
GREENPOINT AV_N405_R239_00-00-01        1
182-183 STS_N215_R237_00-00-02          1
DEKALB AV_H023_R236_00-06-01            1
40 ST LOWERY ST_R518_R261_00-00-00      1
30 AV_R513_R093_00-03-01                1
FLUSHING-MAIN_R533_R055_00-03-07        1
36 AV_R511_R091_00-00-00                1
34 ST-PENN STA_N071_R013_00-06-00       1
30 AV_R513_R093_00-03-02                1
3 AV-149 ST_R311_R053_00-00-03          1
174-175 STS_N212_R253_01-06-00          1
LEXINGTON AV/53_N305A_R016_00-00-02     1
GRD CNTRL-42 ST_R238_R046_00-00-00      1
BROOKLYN BRIDGE_R210_R044_00-03-03      1
149/GRAND CONC_R261_R205_00-00-01       1
FORDHAM RD_R289_R119_00-03-00           1
EASTN PKWY-MUSM_R621_R060_00-03-01      1
167 ST_N206_R104_01-06-00               1
BURNSIDE AV_R287_R244_00-05-00    

#
#identify outlier values of 'ENTRIES' for Station/Date pairs and replace them with weekly averages for the station

In [202]:
high_turnstile_count = dict(by_turnstile_date[by_turnstile_date['ENTRIES']>5000].turnstile_id.value_counts())
high_turnstile_count = {key : val for key,val in high_day_count.items() if val<2}
high_turnstile_count

{'149/GRAND CONC': 1,
 '167 ST': 1,
 '174-175 STS': 1,
 '182-183 STS': 1,
 '183 ST': 1,
 '190 ST': 1,
 '3 AV-149 ST': 1,
 '30 AV': 1,
 '36 AV': 1,
 '40 ST LOWERY ST': 1,
 '57 ST-7 AV': 1,
 'AVENUE H': 1,
 'BROOKLYN BRIDGE': 1,
 'BURNSIDE AV': 1,
 'CASTLE HILL AV': 1,
 'EASTN PKWY-MUSM': 1,
 'FLUSHING-MAIN': 1,
 'FORDHAM RD': 1,
 'GREENPOINT AV': 1,
 'PARK PLACE': 1,
 'WORLD TRADE CTR': 1}

In [203]:
#identify outlier values of 'ENTRIES' for turnstile/Date pairs and replace them with weekly averages for the turnstile
avg_turnstile_per_day = by_turnstile_date.groupby(by=['STATION','turnstile_id','WEEK'], as_index=False).mean().drop(columns='EXITS')
avg_turnstile_per_day.rename(columns={'ENTRIES':'AVG_REPLACEMENT'}, inplace=True)

by_turnstile_date_anomalies = by_turnstile_date[(by_turnstile_date['ENTRIES']>30000) & (by_turnstile_date['turnstile_id'].isin(high_turnstile_count.keys()))]
by_turnstile_date_anomalies = by_turnstile_date_anomalies.merge(right=avg_turnstile_per_day, how='left')
by_turnstile_date_anomalies['ENTRIES'] = by_turnstile_date_anomalies['AVG_REPLACEMENT']
by_turnstile_date_anomalies.drop(columns=['AVG_REPLACEMENT'], inplace=True)

by_turnstile_date = pd.concat([by_turnstile_date[(by_turnstile_date['ENTRIES']<=30000) | ~(by_turnstile_date['turnstile_id'].isin(high_turnstile_count.keys()))], by_turnstile_date_anomalies])

by_turnstile_date.ENTRIES.describe()

count    1.702130e+05
mean     8.142840e+03
std      1.218717e+06
min      0.000000e+00
25%      1.700000e+02
50%      4.540000e+02
75%      8.610000e+02
max      2.791039e+08
Name: ENTRIES, dtype: float64

In [204]:
by_station_date = by_turnstile_date.groupby(by=['STATION','WEEK', 'DATE'], as_index=False).sum()
by_station_date.ENTRIES.describe()

count    1.462800e+04
mean     9.475097e+04
std      4.910535e+06
min      0.000000e+00
25%      1.643750e+03
50%      3.184500e+03
75%      7.165500e+03
max      4.581631e+08
Name: ENTRIES, dtype: float64

In [205]:
#stations with 1 really high day's entries probably are showing data anomalies.
by_station_date[by_station_date['ENTRIES']>30000].STATION.value_counts()

34 ST-HERALD SQ    69
34 ST-PENN STA     64
TIMES SQ-42 ST     60
59 ST COLUMBUS     51
47-50 STS ROCK     50
CHAMBERS ST        49
23 ST              49
59 ST              49
GRD CNTRL-42 ST    49
LEXINGTON AV/53    48
42 ST-BRYANT PK    47
50 ST              40
86 ST              34
JAY ST-METROTEC    21
DEKALB AV           2
WORLD TRADE CTR     1
149/GRAND CONC      1
183 ST              1
PARK PLACE          1
FLUSHING-MAIN       1
57 ST-7 AV          1
30 AV               1
40 ST LOWERY ST     1
36 AV               1
BURNSIDE AV         1
190 ST              1
BROOKLYN BRIDGE     1
AVENUE H            1
FORDHAM RD          1
EASTN PKWY-MUSM     1
CASTLE HILL AV      1
174-175 STS         1
3 AV-149 ST         1
GREENPOINT AV       1
182-183 STS         1
167 ST              1
Name: STATION, dtype: int64

In [206]:
#identify outlier values of 'ENTRIES' for Station/Date pairs and replace them with weekly averages for the station
high_day_count = dict(by_station_date[by_station_date['ENTRIES']>30000].STATION.value_counts())
high_day_count = {key : val for key,val in high_day_count.items() if val<=2}
high_day_count

{'149/GRAND CONC': 1,
 '167 ST': 1,
 '174-175 STS': 1,
 '182-183 STS': 1,
 '183 ST': 1,
 '190 ST': 1,
 '3 AV-149 ST': 1,
 '30 AV': 1,
 '36 AV': 1,
 '40 ST LOWERY ST': 1,
 '57 ST-7 AV': 1,
 'AVENUE H': 1,
 'BROOKLYN BRIDGE': 1,
 'BURNSIDE AV': 1,
 'CASTLE HILL AV': 1,
 'DEKALB AV': 2,
 'EASTN PKWY-MUSM': 1,
 'FLUSHING-MAIN': 1,
 'FORDHAM RD': 1,
 'GREENPOINT AV': 1,
 'PARK PLACE': 1,
 'WORLD TRADE CTR': 1}

In [207]:
#identify outlier values of 'ENTRIES' for Station/Date pairs and replace them with weekly averages for the station
avg_day_per_week = by_station_date.groupby(by=['STATION','WEEK'], as_index=False).mean().drop(columns='EXITS')
avg_day_per_week.rename(columns={'ENTRIES':'AVG_REPLACEMENT'}, inplace=True)

by_station_date_anomalies = by_station_date[(by_station_date['ENTRIES']>30000) & (by_station_date['STATION'].isin(high_day_count.keys()))]
by_station_date_anomalies = by_station_date_anomalies.merge(right=avg_day_per_week, how='left')
by_station_date_anomalies['ENTRIES'] = by_station_date_anomalies['AVG_REPLACEMENT']
by_station_date_anomalies.drop(columns=['AVG_REPLACEMENT'], inplace=True)

by_station_date = pd.concat([by_station_date[(by_station_date['ENTRIES']<=30000) | ~(by_station_date['STATION'].isin(high_day_count.keys()))], by_station_date_anomalies])

by_station_date.ENTRIES.describe()

count    1.462800e+04
mean     2.095341e+04
std      7.236803e+05
min      0.000000e+00
25%      1.643750e+03
50%      3.184500e+03
75%      7.165500e+03
max      6.546071e+07
Name: ENTRIES, dtype: float64

In [208]:
by_station_date.head()

,STATION,WEEK,DATE,ENTRIES,EXITS
0,1 AV,1,04/21/2018,10802.0,11866
1,1 AV,1,04/22/2018,8406.0,9639
2,1 AV,1,04/23/2018,11463.0,10550
3,1 AV,1,04/24/2018,11590.0,11111
4,1 AV,1,04/25/2018,11849.0,11164


In [209]:
by_station_date.groupby(by=['STATION','WEEK'], as_index=False).count().ENTRIES.value_counts()

7    2082
6       6
3       2
5       1
1       1
4       1
2       1
Name: ENTRIES, dtype: int64

In [210]:
by_station_date.ENTRIES.describe()

count    1.462800e+04
mean     2.095341e+04
std      7.236803e+05
min      0.000000e+00
25%      1.643750e+03
50%      3.184500e+03
75%      7.165500e+03
max      6.546071e+07
Name: ENTRIES, dtype: float64

In [211]:
with open('by_station_date_cleaned.pickle','wb') as f:
    pickle.dump(by_station_date,f)

In [212]:
by_station_week =  by_station_date.groupby(by=['STATION', 'WEEK'], as_index=False).sum().drop(columns=['EXITS'])
by_station_week.head()

,STATION,WEEK,ENTRIES
0,1 AV,1,78562.0
1,1 AV,2,79165.0
2,1 AV,3,77357.0
3,1 AV,4,76875.0
4,1 AV,5,79160.0


In [213]:
by_station_week.ENTRIES.describe()

count    2.094000e+03
mean     1.463737e+05
std      1.912969e+06
min      0.000000e+00
25%      1.204325e+04
50%      2.366850e+04
75%      5.193025e+04
max      6.552258e+07
Name: ENTRIES, dtype: float64

In [214]:
by_station_week[by_station_week['ENTRIES']>500000]

,STATION,WEEK,ENTRIES
167,174-175 STS,8,1.378611e+06
229,182-183 STS,10,3.025962e+07
233,183 ST,4,2.490976e+07
247,190 ST,8,2.923168e+07
366,3 AV-149 ST,7,6.552258e+07
370,30 AV,9,2.007817e+06
412,36 AV,9,1.379202e+06
487,47-50 STS ROCK,4,2.186936e+07
566,57 ST-7 AV,3,7.356971e+05
1067,BURNSIDE AV,1,2.416359e+06


In [215]:
#get rid of anomalously high weeks
by_station_week = by_station_week[by_station_week['ENTRIES']<500000]

In [229]:
weekly_avg_by_station = by_station_week.groupby(by="STATION", as_index=False).mean().drop(columns=['WEEK'])
weekly_avg_by_station["mean_percentage"] = (weekly_avg_by_station["ENTRIES"]/weekly_avg_by_station["ENTRIES"].sum())*100
weekly_avg_by_station = weekly_avg_by_station.sort_values(by='ENTRIES', ascending=False)
weekly_avg_by_station.head(10)

,STATION,ENTRIES,mean_percentage
39,34 ST-HERALD SQ,466006.700000,4.504909
200,TIMES SQ-42 ST,370798.555556,3.584527
41,34 ST-PENN STA,329103.500000,3.181459
31,23 ST,291405.100000,2.817027
60,59 ST COLUMBUS,289903.300000,2.802509
59,59 ST,260665.200000,2.519863
143,GRD CNTRL-42 ST,258111.400000,2.495175
115,CHAMBERS ST,250260.400000,2.419279
50,47-50 STS ROCK,247871.000000,2.396181
74,86 ST,204099.200000,1.973037


In [231]:
with open('weekly_avg_by_station_clean.pickle', 'wb') as to_write:
    pickle.dump(weekly_avg_by_station, to_write)

In [227]:
weekly_avg_by_station.head(10)

,STATION,ENTRIES,mean_percentage
0,1 AV,75148.3,0.726462
1,103 ST-CORONA,39958.6,0.386282
2,104 ST,7125.0,0.068878
3,110 ST,32489.1,0.314074
4,111 ST,28965.5,0.280011
5,121 ST,2889.7,0.027935
6,125 ST,122302.6,1.182305
7,135 ST,40519.2,0.391701
8,138/GRAND CONC,9103.9,0.088008
9,14 ST,146881.3,1.419908


In [232]:
weekly_avg_by_station.head(10).mean_percentage.sum()

28.693964807482622